In [1]:
import pandas as pd
import data_util
from tec_an import TecAn
import numpy as np
from data_util import *
from bitstamp import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb


In [2]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = 1234,
                 val_end = 1234,
                 train_start_list = []
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [10]:
win = 40
minus = 15
step =  1

tec = TecAn(windows = win, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(minus, sourceDataGenerator, True, win)

In [11]:
train_x_raw, train_y_raw = online.load_train_data()
train_x, train_y, _closed_prices = get_sequencial_data(train_x_raw, train_y_raw , step)
print(Counter(train_y))

Counter({0: 16198, 1: 7778})


In [5]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1):
    n_vars = 1
    df = pd.DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    
    return agg.fillna(0).values

In [6]:
past = [ i for i in range(3)]

scoring = [
    #'accuracy', 
    'f1', 
    'precision', 
    'recall'
]

score = 'recall'

In [7]:

def getModel():
    param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
    #param = {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 3, 'random_state': 42}
    coModel = RandomForestClassifier(**param)
    return coModel

In [8]:
from sklearn.feature_selection import SelectFromModel
from sklearn.gaussian_process import GaussianProcessClassifier
from catboost import CatBoostClassifier
from sklearn.feature_selection import RFE, RFECV
from pytorch_tabnet.tab_model import TabNetClassifier

def test(score, n_in):
    
    train_x_raw, train_y_raw = online.load_train_data()
    train_x, train_y, _closed_prices = get_sequencial_data(series_to_supervised(train_x_raw, n_in=n_in), train_y_raw , step)
    
    #train_x, train_y = get_balanced_set(train_x, train_y)

    val_x_raw, val_y_raw = online.load_val_data("ltceur")
    val_x, val_y, closed_prices = get_sequencial_data(series_to_supervised(val_x_raw, n_in=n_in), val_y_raw , step)


    svc = DecisionTreeClassifier(random_state=10)
    #svc = RandomForestClassifier(random_state=10)
    #svc = xgb.XGBClassifier(random_state=10)
    
    
    clf = TabNetClassifier(verbose=0) 
    
    
    cls = xgb.XGBClassifier(random_state=1,
                                       objective = "binary:logistic",
                                       eval_metric='mlogloss',
                                       learning_rate=0.01, 
                                       #use_label_encoder = True
                           )
    
    cls = make_pipeline(StandardScaler(),
                                MLPClassifier(
                                    solver='lbfgs', alpha=1, random_state=1, max_iter=20000,
                                    early_stopping=True)
                                )
    
    cls = CatBoostClassifier(logging_level = 'Silent')

    
    rfecv = RFECV(estimator=svc, scoring=score, min_features_to_select=15)
    #rfecv = RFECV(estimator=svc)
    #rfecv = SelectFromModel(estimator=svc)
    #rfecv = RFE(estimator=RandomForestClassifier(random_state=10), n_features_to_select=10)

    pipe = Pipeline(
                    steps=[
                    #('s',rfecv),
                    ('m',cls)
                ])
    
    pipe.fit(train_x, train_y)
    

    back = BackTest(value = 100, 
                verbose = False, 
                sell_on_profit = True,
                pending_sell_steps = step)

    back = backtest_model(pipe, val_x, closed_prices, back)

    print(f"{score} - {back.current}")
    
    return rfecv

In [9]:
for p in past:
    print(f"Past {p}")
    test(score, p)

Past 0


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-a63cc69f1b9b>", line 3, in <module>
    test(score, p)
  File "<ipython-input-8-238de30aea0b>", line 53, in test
    pipe.fit(train_x, train_y)
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/catboost/core.py", line 4673, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/catboost/core.py", line 1994, in _fit
    self._train(
  File "/Users/samirantonio/opt/

TypeError: object of type 'NoneType' has no len()

In [ ]:
Counter({0: 18422, 1: 5554}) 106.72623575802629

In [ ]:
for p in past:
    print(f"Past {p}")
    #test(score, p)
    for score in scoring:
        rfecv = test(score, p)
        #break
        

In [ ]:
preds = pipe.predict(xx)

In [ ]:
back = BackTest(value = 100, 
                verbose = True, 
                sell_on_profit = True,
                pending_sell_steps = step)


#backtest_baseline(x, y, closed_prices, step, back)
backtest_baseline(x, preds, closed_prices, step, back)

print(back.current)

In [ ]:
print(rfecv)
min_features_to_select = rfecv.min_features_to_select
print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

In [ ]:
pd.DataFrame(rfecv.support_)

In [ ]:
 datetime.datetime.fromtimestamp(1626174900)

In [15]:
def load_online(minutes, window, val_end, currency = "btcusd"):
    tec = TecAn(windows = window, windows_limit = 100)
    source_data_generator = SourceDataGenerator(tec = tec)


    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 40,
                 val_limit = 1000,
                 val_keys = [currency],
                 val_start = val_end - (60),
                 val_end = val_end,
                 train_start_list = []
    )

    start = val_end - (60 * 100 * minutes)
    end = val_end - (60 * minutes)

    online.load_val_cache(
                    val_keys = [currency],                  
                    start = start,
                    end = end)
    return online

In [16]:
load_online(15, 30, 1627217023)

https://www.bitstamp.net/api/v2/ohlc/btcusd/?&currency_pair=btcusd&step=900&limit=1000&start=1627199023
Downloaded from 2021-07-25 08:30:00 to 2021-07-25 13:30:00 34347.59
Resample 15Min - TecAn ( windows 30, windows_limit 100 )
Processing 20 of btcusd


  0%|          | 0/20 [00:00<?, ?it/s]